In [1]:
import pandas as pd
import numpy as np
import os
import re
import time

import requests
from bs4 import BeautifulSoup

from scrap_utils import Quote, convert_manual_author, extrat_quote, tokenize, get_babelio_citation, scrap_recent_babelio
%load_ext autoreload
%autoreload 2

In [22]:
# read epub
import ebooklib
from ebooklib import epub

# open epub
book = epub.read_epub('data_v1.1/epub/Le Gai Savoir (avec Dossier) -- Friedrich Nietzsche, Patrick Wotling -- GF, 2020 -- Flammarion -- 9782080208897 -- 33445904880eb0d6db6c7b6a44bba438 -- Anna’s Archive.epub')

# get all the text
text = ''
for item in book.get_items():
    if item.get_type() == ebooklib.ITEM_DOCUMENT:
        text += item.get_body_content().decode('utf-8')
text

# clean text
text = re.sub(r'<.*?>', '', text)
# export text into .txt file
with open('data_v1.1/text_from_epub/Le Gai Savoir.txt', 'w') as f:
    pass
    # f.write(text)



/Users/t.schmitt/miniconda3/lib/python3.10/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/Users/t.schmitt/miniconda3/lib/python3.10/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [31]:
# read file
with open('data_v1.1/text_from_epub/Gai Savoir - ruse.txt', 'r') as f:
    text = f.read()

list_aphorism = []
num = -1
aphorisme = ''
titre = ''
for line in text.split('\n'):
    # first char is a number
    if line and line[0] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
        num_titre = line
        num = line.split(' ')[0]
        titre = ' '.join(line.split(' ')[1:])
    else:
        if line != '':
            aphorisme += line + ' '
        else:
            if len(aphorisme) > 2:
                list_aphorism.append([num, titre, aphorisme])
                aphorisme = ''

df = pd.DataFrame(list_aphorism, columns=['num', 'titre', 'aphorisme'])
df.to_csv('data_v1.1/text_from_epub/Gai Savoir - ruse.csv', index=False)

In [2]:
# read file
with open('data_v1.1/text_from_epub/Gai savoir livre1_5.txt', 'r') as f:
    text = f.read()

list_aphorism = []
num = -1
aphorisme = ''
titre = ''
for line in text.split('\n'):
    if len(line) < 1:
        continue
    # first char is a number
    if line and line[0] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
        num = line
    else:
        if len(line) > 2:
            # split text by sentence
            index_p = line.find('.')
            if index_p == -1:
                index_p = len(line)
            index_pi = line.find('?')
            if index_pi == -1:
                index_pi = len(line)
            index_pe = line.find('!')      
            if index_pe == -1:
                index_pe = len(line)
            index = min([index_p, index_pi, index_pe])
            index += 1
            titre = line[:index]
            aphorism = line[index+2 :]
            list_aphorism.append([num, titre, aphorism])
        # break
df = pd.DataFrame(list_aphorism, columns=['num', 'titre', 'aphorisme'])
df.to_csv('data_v1.1/text_from_epub/Gai savoir livre1_5.csv', index=False)
df.iloc[-4:]

,num,titre,aphorisme
379,380,« Le voyageur » parle.,Pour considérer notre moralité européenne de ...
380,381,Éléments pour la question de l'intelligibilité.,"On ne veut pas seulement être compris, quand ..."
381,382,La grande santé.,"Nous, nouveaux, sans-nom, difficiles à compre..."
382,383,Épilogue.,Mais tandis qu'en conclusion je peins très tr...


In [3]:
df_livre = pd.read_csv('data_v1.1/text_from_epub/Gai savoir livre1_5.csv')
df_livre["book"] = "Le Gai Savoir"
df_ruse = pd.read_csv('data_v1.1/text_from_epub/Gai Savoir - ruse.csv')
df_ruse["book"] = "Le Gai Savoir (ruse)"
df = pd.concat([df_livre, df_ruse])
# remove digits
df['quote'] = df['aphorisme'].str.replace(r'\d', '')
df['titre'] = df['titre'].str.replace(r'\d', '')
df['author'] = 'Friedrich Nietzsche'
df['title'] = df['num'].astype('str') + ' - ' + df['titre']
df.drop(columns=['num', 'titre', 'aphorisme'], inplace=True)
df.to_csv('data_v1.1/text_from_epub/Le Gai Savoir.csv', index=False)
df

/var/folders/fg/s665ygqj6gq0l21kgx9k0rxm0000gp/T/ipykernel_80910/2222822344.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['quote'] = df['aphorisme'].str.replace(r'\d', '')
/var/folders/fg/s665ygqj6gq0l21kgx9k0rxm0000gp/T/ipykernel_80910/2222822344.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['titre'] = df['titre'].str.replace(r'\d', '')


,book,quote,author,title
0,Le Gai Savoir,Que je considère les hommes d'un œil bon ou m...,Friedrich Nietzsche,1 - Les théoriciens du but de l'existence.
1,Le Gai Savoir,Je ne cesse de refaire la même expérience et ...,Friedrich Nietzsche,2 - La conscience morale en matière intellectu...
2,Le Gai Savoir,"Tous les sentiments nobles, généreux paraisse...",Friedrich Nietzsche,3 - Noble et vulgaire.
3,Le Gai Savoir,Ce sont les esprits les plus forts et les plu...,Friedrich Nietzsche,4 - Ce qui conserve l'espèce.
4,Le Gai Savoir,Tous les hommes qui sentent qu'ils ont besoin ...,Friedrich Nietzsche,5 - Devoirs inconditionnés.
...,...,...,...,...
60,Le Gai Savoir (ruse),La plume griffonne : quel enfer ! Suis-je cond...,Friedrich Nietzsche,59 - La plume griffonne
61,Le Gai Savoir (ruse),Celui-ci s'élève – il faut le louer ! Mais cel...,Friedrich Nietzsche,60 - Hommes supérieurs
62,Le Gai Savoir (ruse),"La moitié de ta vie est écoulée, L'aiguille av...",Friedrich Nietzsche,61 - Le sceptique parle
63,Le Gai Savoir (ruse),Oui ! Je connais mon ascendance ! Insatiable t...,Friedrich Nietzsche,62 - Ecce Homo


In [35]:
df = pd.read_csv('data_v1.1/text_from_epub/Le Gai Savoir.csv')
df['text_tok'] = df['quote'].astype('str').apply(tokenize)
df['text'] = df['quote']
df['nb_like'] = 1
df['haiku'] = False
df['quote_react'] = None
df['note'] = None
df['all_search'] = (df['text_tok'].astype(str)) + (df['author'].astype(str).apply(tokenize)) + (df['book'].astype(str).apply(tokenize)) + (df['title'].astype(str).apply(tokenize)) + df['quote_react'].astype(str)
df['is_delete'] = False
df.drop(columns=['quote'], inplace=True)
df

,book,author,title,text_tok,text,nb_like,haiku,quote_react,note,all_search,is_delete
0,Le Gai Savoir,Friedrich Nietzsche,1 - Les théoriciens du but de l'existence.,que je considere les hommes d un oeil bon ou m...,Que je considère les hommes d'un œil bon ou m...,1,False,None,None,que je considere les hommes d un oeil bon ou m...,False
1,Le Gai Savoir,Friedrich Nietzsche,2 - La conscience morale en matière intellectu...,je ne cesse de refaire la meme experience et n...,Je ne cesse de refaire la même expérience et ...,1,False,None,None,je ne cesse de refaire la meme experience et n...,False
2,Le Gai Savoir,Friedrich Nietzsche,3 - Noble et vulgaire.,tous les sentiments nobles genereux paraissent...,"Tous les sentiments nobles, généreux paraisse...",1,False,None,None,tous les sentiments nobles genereux paraissent...,False
3,Le Gai Savoir,Friedrich Nietzsche,4 - Ce qui conserve l'espèce.,ce sont les esprits les plus forts et les plus...,Ce sont les esprits les plus forts et les plu...,1,False,None,None,ce sont les esprits les plus forts et les plus...,False
4,Le Gai Savoir,Friedrich Nietzsche,5 - Devoirs inconditionnés.,tous les hommes qui sentent qu ils ont besoin ...,Tous les hommes qui sentent qu'ils ont besoin ...,1,False,None,None,tous les hommes qui sentent qu ils ont besoin ...,False
...,...,...,...,...,...,...,...,...,...,...,...
443,Le Gai Savoir (ruse),Friedrich Nietzsche,59 - La plume griffonne,la plume griffonne quel enfer suis je condamne...,La plume griffonne : quel enfer ! Suis-je cond...,1,False,None,None,la plume griffonne quel enfer suis je condamne...,False
444,Le Gai Savoir (ruse),Friedrich Nietzsche,60 - Hommes supérieurs,celui ci s eleve il faut le louer mais celui l...,Celui-ci s'élève – il faut le louer ! Mais cel...,1,False,None,None,celui ci s eleve il faut le louer mais celui l...,False
445,Le Gai Savoir (ruse),Friedrich Nietzsche,61 - Le sceptique parle,la moitie de ta vie est ecoulee l aiguille ava...,"La moitié de ta vie est écoulée, L'aiguille av...",1,False,None,None,la moitie de ta vie est ecoulee l aiguille ava...,False
446,Le Gai Savoir (ruse),Friedrich Nietzsche,62 - Ecce Homo,oui je connais mon ascendance insatiable telle...,Oui ! Je connais mon ascendance ! Insatiable t...,1,False,None,None,oui je connais mon ascendance insatiable telle...,False


In [32]:
# archive from data_v1 (i.e. all scrapping)
data_v1 = pd.read_parquet('data_v2/raw_data_v1_17_nov_.parquet') 
data_v1['note'] = data_v1['vo']
data_v1['all_search'] = (data_v1['text_tok'].astype(str)) + (data_v1['author'].astype(str).apply(tokenize)) + (data_v1['book'].astype(str).apply(tokenize)) + (data_v1['title'].astype(str).apply(tokenize)) + data_v1['quote_react'].astype(str)
data_v1.set_index('text_tok', inplace = True, drop = True)
data_v1.drop(['year', 'source', 'confiance', 'page', 'url', 'nb_char', 'nb_lines', 'sonnet', 'vo'], axis = 1, inplace = True)
data_v1['is_delete'] = False
data_v1

,text,author,book,title,nb_like,haiku,quote_react,note,all_search,is_delete
text_tok,,,,,,,,,,
celui qui passe a cote de la plus belle histoire de sa vie n aura que l age de ses regrets et tous les soupirs du monde ne sauraient bercer son ame,Celui qui passe à côté de la plus belle histoi...,Yasmina Khadra,Ce que le jour doit à la nuit,nan,2740.0,False,None,nan,celui qui passe a cote de la plus belle histoi...,False
il est tellement important de laisser certaines choses disparaitre de s en defaire de s en liberer il faut comprendre que personne ne joue avec des cartes truquees parfois on gagne parfois on perd n attendez pas que l on vous rende quelque chose n attendez pas que l on comprenne votre amour vous devez clore des cycles non par fierte par orgueil ou par incapacite mais simplement parce que ce qui precede n a plus sa place dans votre vie faites le menage secouez la poussiere fermez la porte changez de disque cessez d etre ce que vous etiez et devenez ce que vous etes,Il est tellement important de laisser certaine...,Paulo Coelho,Le Zahir,nan,2270.0,False,None,nan,il est tellement important de laisser certaine...,False
je me sens toujours heureux savez vous pourquoi parce que je n attends rien de personne les attentes font toujours mal la vie est courte aimez votre vie soyez heureux gardez le sourire et souvenez vous avant de parler ecoutez avant d ecrire reflechissez avant de prier pardonnez avant de blesser considerez l autre avant de detester aimez et avant de mourir vivez,"Je me sens toujours heureux, savez vous pourqu...",William Shakespeare,None,nan,2215.0,False,None,nan,je me sens toujours heureux savez vous pourquo...,False
l esclave qui n est pas capable d assumer sa revolte ne merite pas que l on s apitoie sur son sort cet esclave repondra seul de son malheur s il se fait des illusions sur la condescendance suspecte d un maitre qui pretend l affranchir seule la lutte libere,L'esclave qui n'est pas capable d'assumer sa r...,Thomas Sankara,None,nan,2013.0,False,None,nan,l esclave qui n est pas capable d assumer sa r...,False
pourquoi croise t on des milliers de personnes et ne s eprend on que d une seule,Pourquoi croise-t-on des milliers de personnes...,Guillaume Musso,Sauve-moi,nan,1912.0,False,None,nan,pourquoi croise t on des milliers de personnes...,False
...,...,...,...,...,...,...,...,...,...,...
avec ma mere c etait toujours la guerre froide j etais capable d amour pour presque tous les humains et les etres vivants il n y avait qu une seule personne que je n arrivais pas a aimer sincerement ma mere mon antipathie pour elle etait profonde et massive presque autant que l energie qui me faisait aimer tout le reste voila qui j etais vraiment,Avec ma mère c'était toujours la guerre froide...,Ito Ogawa,Le restaurant de l'amour retrouvé,nan,0.0,False,None,nan,avec ma mere c etait toujours la guerre froide...,False
ma mesentente avec ma mere etait precisement cette boue en moi mais si je demeurais sereine elle ne salirait pas tout mon coeur donc je faisais en sorte d eviter ma mere le plus possible en un sens je m appliquais a ignorer sa presence j etais convaincue que c etait la le seul moyen de garder le coeur pur,Ma mésentente avec ma mère était précisément c...,Ito Ogawa,Le restaurant de l'amour retrouvé,nan,0.0,False,None,nan,ma mesentente avec ma mere etait precisement c...,False
la main que j avais retiree de celle de la jeune fille etait toute moite exactement comme si je tenais serrees dans mon poing les larmes qui coulaient sur ses joues le vacarme des trains qui passaient etouffait le son de ses sanglots mais je suis sur qu elle pleurait,La main que j'avais retirée de celle de la jeu...,Ito Ogawa,Le Jardin Arc-en-ciel,nan,0.0,False,None,nan,la main que j avais retiree de celle de la jeu...,False


In [36]:
len("""Oeuvres philosophiques complètes, tome 5 : Le gai""")

49